### Panda Analysis

In [1]:
# some import statments
import os
import glob
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tnrange
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Conv2D, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
path='./data_sets/pandaml/'

In [ ]:
# Sequential (Hit_Predictor)
def init_model_sequential():
    model = Sequential()
    model.add(Bidirectional(LSTM(200, return_sequences=True), input_shape=(None, 2)))
    model.add(Bidirectional(LSTM(100, return_sequences=False)))
    model.add(RepeatVector(1))
    model.add(TimeDistributed(Dense(2)))
    return model

# Functional (Hit_Predictor)
def init_model_functional():
    inputs = Input(shape=(None,2))
    h1 = Bidirectional(LSTM(200, return_sequences=True))(inputs)
    h2 = Bidirectional(LSTM(100, return_sequences=False))(h1)
    h3 = RepeatVector(1)(h2)
    outputs = TimeDistributed(Dense(2))(h3)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Current Model
def init_model(features=None):
    model = Sequential()
    model.add(Dense(400, activation='tanh', input_shape=(features,), kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(300, activation='tanh', kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(200, activation='tanh', kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='tanh', kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='tanh', kernel_initializer='glorot_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model = init_model(features=X.shape[1])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=2048, validation_split=0.10)

In [ ]:
model.save('pandaml.h5')

### Plotting:

In [ ]:
# For validation accuracy > training accuracy, for reasons see;
# https://stackoverflow.com/questions/43979449/higher-validation-accuracy-than-training-accurracy-using-tensorflow-and-keras
# https://keras.io/getting-started/faq/#why-is-the-training-loss-much-higher-than-the-testing-loss
# http://cs231n.github.io/neural-networks-3/#sanitycheck

hist = history.history

plt.style.use('ggplot')
fig = plt.figure(figsize=(20,10))

plt.subplot(1, 2, 1)
plt.plot(np.array(hist['acc']), label='Training',  color='blue', linewidth= 2)
plt.plot(np.array(hist['val_acc']), label='Validation',  color='red', linewidth= 2)
plt.xlim(0)
plt.ylim(0.5,1.00)
plt.grid(True)
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.legend(loc='best', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('train_before_skewed')

plt.subplot(1, 2, 2)
plt.plot(np.array(hist['loss']), label='Training',  color='blue', linewidth= 2)
plt.plot(np.array(hist['val_loss']), label='Validation',  color='red', linewidth= 2)
plt.xlim(0)
plt.ylim(0.0,0.5)
plt.grid(True)
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Loss', fontsize=15)
plt.legend(loc='best', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.title('train_all')
plt.savefig('train_all.png')
plt.show()